In [1]:
import tensorflow as tf
import cv2
import time
from myprotos import string_int_label_map_pb2
from google.protobuf import text_format

#讀取 Label Map
def LoadLabelmap(LabelMapPath , UseDisplayName = True):
    with open(LabelMapPath) as f:
        Labelmap_String = f.read()
    LabelMap = string_int_label_map_pb2.StringIntLabelMap()
    text_format.Merge(Labelmap_String, LabelMap)
    LabelmapDict = {}
    for item in LabelMap.item:
        if (UseDisplayName == True):
            LabelmapDict[item.id] = item.display_name 
        else:
            LabelmapDict[item.id] = item.name 
    return LabelmapDict

MyCam = cv2.VideoCapture('ValidateVideo.mp4') #使用影片物件檢測，改成Mcv2.VideoCapture(0)可以使用攝影機
LabelmapDict = LoadLabelmap(r'Model/ForClass_SSDMobilenetV2_OID/labelmap.pbtxt',False)
#讀取模型
model = tf.saved_model.load(r'Model/ForClass_SSDMobilenetV2_OID')
model = model.signatures['serving_default']
#讀取圖型
while(MyCam.isOpened()):
    StartTime = time.time()
    ret, cvimg = MyCam.read()
    if (ret == False): #影片讀取結束或是讀取錯誤，所以讀取失敗
        break
    cvimgRGB = cv2.cvtColor(cvimg, cv2.COLOR_BGR2RGB)
    #使用模型推理
    input_tensor = tf.convert_to_tensor(cvimgRGB)
    input_tensor = input_tensor[tf.newaxis,...]
    out = model(input_tensor)
    #繪出結果
    # 檢測數量         = out['num_detections'][第幾張圖片]
    # 檢測物件的類別    = out['detection_classes'][第幾張圖片][第幾個框檢測結果] 
    # 檢測物件信心指數  = out['detection_scores'][第幾張圖片][第幾個框檢測結果] 
    # 檢測物件的框座標  = out['detection_boxes'][第幾張圖片][第幾個檢測結果][4個座標]   4個座標分別為:0起始座標y , 1起始座標x , 2終點座標y , 3終點座標x
    for i in range(int(out['num_detections'][0])):
        ClassID = int(out['detection_classes'][0][i])
        Score = float(out['detection_scores'][0][i])
        if (Score > 0.5):
            y1 = out['detection_boxes'][0][i][0] * cvimgRGB.shape[0]
            x1 = out['detection_boxes'][0][i][1] * cvimgRGB.shape[1]
            y2 = out['detection_boxes'][0][i][2] * cvimgRGB.shape[0]
            x2 = out['detection_boxes'][0][i][3] * cvimgRGB.shape[1]
            cv2.rectangle(cvimg,(x1,y1),(x2,y2),(0,255,0),thickness=3)
            info = 'Class:{} Score:{}'.format(LabelmapDict[ClassID], round(Score,2))   
            #cv2.putText(目標圖片,   文字,   (x,y),字型             ,字體大小縮放,顏色(B,G,R),字體粗細))
            cv2.putText(cvimg, info,(x1+5,y1+30), cv2.FONT_HERSHEY_COMPLEX,1, (255, 255, 0), 2)
    EndTime = time.time()
    Dtime = EndTime - StartTime
    cv2.putText(cvimg, str(int(Dtime*1000)) + 'ms', (0 + 5, 0 + 30), cv2.FONT_HERSHEY_COMPLEX,1, (0, 255, 255), 1)
    cv2.imshow('Hello',cvimg) 
    if cv2.waitKey(1) == ord('q'):
                break
MyCam.release()
cv2.destroyAllWindows()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
